In [9]:
# Generate Data
def GenerateData(select):
    random.seed(100)
    if select == 1:
        CL1 = 7
        CL2 = 7
        data = np.zeros((14,2))
        for i in range(1,CL1+1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1+1,CL2+CL1+1):
            data[i,0] = 3 + random.random()
            data[i,1] = 3 + random.random()
    elif select == 2:
        CL1 = 7
        CL2 = 7
        CL3 = 7
        data = np.zeros((21,2))
        for i in range(1,CL1+1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1+1,CL2+CL1+1):
            data[i,0] = 3 + random.random()
            data[i,1] = 1 + random.random()

        for i in range(CL2+CL1+1,CL3+CL2+CL1+1):
            data[i,0] = 1.5 + random.random()
            data[i,1] = 3 + random.random()
    return data

In [8]:
def CalculateAffinity(data):
    sigma = 1
    col_size = data.shape[0]
    affinity = np.zeros((col_size,col_size))
    for i in range(col_size):
        for j in range(col_size):
            dist = math.sqrt((data[i,0]-data[j,0])^2 + (data[i, 1] - data[j,1])^2)
            affinity[i,j] = math.exp(-dist/(2*sigma^2))
    return affinity